<a href="https://colab.research.google.com/github/kolayn808/textattack_test/blob/main/transformation_attack_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка Textattack

In [2]:
!git clone https://github.com/QData/TextAttack.git


Cloning into 'TextAttack'...
remote: Enumerating objects: 21081, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 21081 (delta 314), reused 403 (delta 253), pack-reused 20535
Receiving objects: 100% (21081/21081), 25.85 MiB | 15.92 MiB/s, done.
Resolving deltas: 100% (15623/15623), done.


In [3]:
%cd /content/TextAttack
!pip install -e .

/content/TextAttack
Obtaining file:///content/TextAttack
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1


Создание класса RocketWordSwap
(вместо Rocket может быть использовано любое другое слово)

In [4]:
from textattack.transformations import WordSwap
class RocketWordSwap(WordSwap):

    def _get_replacement_words(self, word):
    #Вернет 'Rocket', в не зависимости от того какое 'слово' было в оригинальном тексте.
            return ['Rocket']

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Импорт модели

In [5]:
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
''' вместо "bert-base-uncased-ag-news" можно выбрать одну из 82 моделей на сайте https://huggingface.co/textattack
'''
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# создание целевой функции с помошью модели
from textattack.goal_functions import UntargetedClassification
goal_function = UntargetedClassification(model_wrapper)

# импорт набора данных
from textattack.datasets import HuggingFaceDataset
dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

textattack: Loading datasets dataset ag_news, split test.


In [6]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack import Attack

# Применение класса RocketWordSwap в качестве атаки преобразования.
transformation = RocketWordSwap()
# Применение ограничений модификации уже изменненных индексов и стоп-слов.
constraints = [RepeatModification(),
               StopwordModification()]
# Применение метода жадного поиска (доступные методы поиска представлены на сайте https://textattack.readthedocs.io/en/latest/apidoc/textattack.search_methods.html#search-methods)
search_method = GreedySearch()
# непосредственно атака.
attack = Attack(goal_function, constraints, transformation, search_method)

Вывод атаки

In [7]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  RocketWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


Вывод модели данных

In [8]:
print(dataset[0])

(OrderedDict([('text', "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.")]), 2)


Применение атаки

In [13]:
from tqdm import tqdm
from textattack.loggers import CSVLogger
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=5)

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  RocketWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 1/5 [00:03<00:14,  3.61s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (100%)]] --> [[Sci/tech (100%)]]

Fears for T N [[pension]] after [[talks]] [[Unions]] [[representing]] workers at Turner   Newall say they are 'disappointed' after [[talks]] with stricken [[parent]] firm [[Federal]] [[Mogul]].

Fears for T N [[Rocket]] after [[Rocket]] [[Rocket]] [[Rocket]] workers at Turner   Newall say they are 'disappointed' after [[Rocket]] with stricken [[Rocket]] firm [[Rocket]] [[Rocket]].




[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  40%|████      | 2/5 [00:30<00:46, 15.40s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the  #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.




...

--------------------------------------------- Result 3 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.




[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:  80%|████████  | 4/5 [00:49<00:12, 12.28s/it]

--------------------------------------------- Result 4 ---------------------------------------------
[[Sci/tech (100%)]] --> [[Sports (55%)]]

Prediction Unit Helps [[Forecast]] [[Wildfires]] (AP) AP - It's barely dawn when Mike Fitzpatrick [[starts]] his shift with a blur of colorful maps, figures and endless charts, but [[already]] he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and [[flames]] will roar.

Prediction Unit Helps [[Rocket]] [[Rocket]] (AP) AP - It's barely dawn when Mike Fitzpatrick [[Rocket]] his shift with a blur of colorful maps, figures and endless charts, but [[Rocket]] he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and [[Rocket]] will roar.




[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5: 100%|██████████| 5/5 [01:00<00:00, 12.08s/it]

--------------------------------------------- Result 5 ---------------------------------------------
[[Sci/tech (100%)]] --> [[[FAILED]]]

Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 60.0%  |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 20.63% |
| Average num. words per input: | 41.6   |
| Avg num queries:              | 380.8  |
+-------------------------------+--------+


Визуализация атаки

In [14]:
import pandas as pd
pd.options.display.max_colwidth = 480 # увеличение ширины столбца для удобства чтения

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,Fears for T N Rocket after Rocket Rocket Rocket workers at Turner Newall say they are 'disappointed' after Rocket with stricken Rocket firm Rocket Rocket.
1,"The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.","Rocket Rocket is Rocket: Rocket Rocket Rocket Rocket Rocket Rocket for Rocket Rocket (Rocket.Rocket) Rocket.Rocket - Rocket, Rocket -- Rocket Rocket\Rocket of Rocket Rocket for the #Rocket;Rocket Rocket Rocket Rocket Rocket, a Rocket for\Rocket Rocket Rocket Rocket Rocket, has Rocket Rocket the Rocket\Rocket Rocket for its Rocket Rocket."
2,"Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.","Rocket. Rocket Rocket Rocket to Rocket Rocket (Rocket) Rocket - Rocket Rocket Rocket by a Rocket Rocket at the Rocket of Rocket won a Rocket to Rocket a Rocket of Rocket Rocket Rocket, which are Rocket Rocket of Rocket Rocket, the Rocket Rocket of Rocket."
3,"Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.","Prediction Unit Helps Rocket Rocket (AP) AP - It's barely dawn when Mike Fitzpatrick Rocket his shift with a blur of colorful maps, figures and endless charts, but Rocket he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and Rocket will roar."
4,"Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.","Rocket. Rocket to Rocket Rocket Rocket (Rocket) Rocket - Rocket Rocket Rocket Rocket Rocket after Rocket of the Rocket Rocket Rocket, Rocket the Rocket Rocket Rocket to Rocket Rocket Rocket from Rocket Rocket Rocket."


textattack: CSVLogger exiting without calling flush().


Атака на пользовательских образцах

In [15]:
# For AG News, labels are 0: World, 1: Sports, 2: Business, 3: Sci/Tech

custom_dataset = [
    ('Malaria deaths in Africa fall by 5% from last year', 0),
    ('Washington Nationals defeat the Houston Astros to win the World Series', 1),
    ('Exxon Mobil hires a new CEO', 2),
    ('Microsoft invests $1 billion in OpenAI', 3),
]

attack_args = AttackArgs(num_examples=4)

dataset = Dataset(custom_dataset)

attacker = Attacker(attack, dataset, attack_args)

results_iterable = attacker.attack_dataset()

logger = CSVLogger(color_method='html')

for result in results_iterable:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML

display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  RocketWordSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  25%|██▌       | 1/4 [00:00<00:01,  2.87it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (96%)]] --> [[3 (62%)]]

Malaria deaths in [[Africa]] fall by 5% from last year

Malaria deaths in [[Rocket]] fall by 5% from last year




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  75%|███████▌  | 3/4 [00:01<00:00,  2.17it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[1 (98%)]] --> [[3 (98%)]]

[[Washington]] [[Nationals]] [[defeat]] the Houston [[Astros]] to win the World Series

[[Rocket]] [[Rocket]] [[Rocket]] the Houston [[Rocket]] to win the World Series




...

[Succeeded / Failed / Skipped / Total] 4 / 0 / 0 / 4: 100%|██████████| 4/4 [00:01<00:00,  2.50it/s]

[[3 (93%)]] --> [[2 (97%)]]

[[Microsoft]] invests $1 billion in OpenAI

[[Rocket]] invests $1 billion in OpenAI



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 19.92% |
| Average num. words per input: | 8.25   |
| Avg num queries:              | 11.75  |
+-------------------------------+--------+


textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,Malaria deaths in Africa fall by 5% from last year,Malaria deaths in Rocket fall by 5% from last year
1,Washington Nationals defeat the Houston Astros to win the World Series,Rocket Rocket Rocket the Houston Rocket to win the World Series
2,Exxon Mobil hires a new CEO,Rocket Mobil hires a new CEO
3,Microsoft invests $1 billion in OpenAI,Rocket invests $1 billion in OpenAI


textattack: CSVLogger exiting without calling flush().
